# Importazione dati
Questo notebook verrà usato per l'importazione di tutti i database che ci servono \
I dati saranno salvati quindi nella cartella data\processed

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import json
from shapely.geometry import Point, Polygon

#Per usare multi cartelle (cookiecutter)
import sys
sys.path.append('./../src')

import make_dataset as m_d




## Grid data

In [ ]:
#GRID
grid=m_d.safe_import("grid")

grid.plot('cellId')
grid

## Non da' problemi, poi dovrò mergiarlo

# Social Pulse

In [ ]:
#Questo fallisce ad importare, come mostrato a lezione 24

tweets_json = json.load( open(m_d.data_path_in / m_d.files['twitter'][0]) )

tweets = gpd.GeoDataFrame(tweets_json['features'])

#Creiamo il punto smontando la casella point
tweets['geometry'] = tweets['geomPoint.geom'].apply(lambda x:Point(x['coordinates'][0], x['coordinates'][1]))
tweets.drop(columns=['geomPoint.geom'],inplace=True)

#Droppo roba inutile
tweets.drop(columns=['municipality.acheneID'],inplace=True)
tweets.drop(columns=['entities'],inplace=True)

tweets.plot("municipality.name")

#tweets.dtypes
tweets

# Weather

In [ ]:
"""
#Fallisce ad importare
weather=m_d.safe_import("weather")
weather
#Come prima ho metadati
"""
weather_json = json.load( open(m_d.data_path_in / m_d.files['weather'][0]) )
weather = gpd.GeoDataFrame(weather_json['features'])


#Elimino le colonne del vento (dati molto incompleti)
weather.drop(weather.columns[list(range(202,298))], axis=1, inplace=True)
weather.drop(columns=['minWind', "maxWind"], inplace=True)

#Svolgiamo infine i punti geometrici
weather['geometry'] = weather['geomPoint.geom'].apply(lambda x:Point(x['coordinates'][0], x['coordinates'][1]))
weather.drop(columns=['geomPoint.geom'],inplace=True)

pd.set_option('display.max_columns', None)

#Lo salvo che mi serve in sezione ML
weather.to_csv(m_d.data_path_out / 'weather_final.csv',index=False)

weather.head(5)



In [ ]:
ax = grid.plot(color="orange")
weather.plot(column='maxTemperature',ax=ax)

#Visto che i tweets sono disassociati dalle stazioni, risulta necessario restringersi alla stazione più vicina
# -> Creo database delle stazioni
stations=m_d.orderstation(weather)


"""
Rimane un problema, le temperature non sono svolte
Spoiler: svolgerle è molto difficile (pandas ha davvero forte limitazioni sulle azioni che fa fare)
         le soluzioni sono molto non-pythoniche (o almeno quelle che ho trovato)
Idea migliore: creo una funzione (timesearch) che svolge il tempo per fare in modo che mi trovi temperatura e precipitazioni
"""
stations.head(5)
print("Mappa della distibuzione della stazioni sul territorio trentino")

# Precipitazioni
Questa tabella è **probabilmente** più precisa di weather (fornisce per ogni elemento della griglia) -> darebbe precipitazione più precisa, ma mancano molte entries. Per ora usiamo solo quello del weather (anche perchè ho già scritto tutto con quel database) \
NOTA: per far questo basta un merge con DB tweets, ma devo ricalcolare il valore alle mezz'ore a causa di come ho discretizzato sotto

In [ ]:
## Lascio qua la cella a puro scopo di chiarezza
"""
Questo si comporta bene
Non è utile importare i data availability tendenzialmente
La seconda colonna è il cellId, non viene detto ma si può verificare con il database grid
    suppongo la terza colonna sia intensità
"""

"""
dummy=m_d.safe_import('precip')
#Problema: il primo dato vien preso come dizionario
#Converto colonne
dummy=dummy.rename(columns={'201311010000': 'time', "2383": 'cellId', "1" : 'intensity'})

#Riaggiungo il primo dato
precipitation=m_d.appforth(dummy,[201311010000,2383,1])

#Converto il tempo per averlo in formato identico agli altri
temp=np.array(precipitation["time"])
#Questi sono integers, li svolgo in modo diverso
minutes=temp%100
temp=np.floor(temp/100)
precipitation["hours"]=np.around(temp%100+minutes/60,2)
temp=np.floor(temp/100)
precipitation["day"]=[int(i%100) for i in temp]
temp=np.floor(temp/100)
precipitation["month"]=[int(i%100) for i in temp]
precipitation.drop(columns=["time"], inplace=True)

precipitation
"""
""""""

# Electro

In [ ]:
# devo prima unire i due dataset di line set -> nov e poi nov -> dicembre

#Problema: il primo dato vien preso come dizionario
electro = m_d.safe_import('SET-1')
electro=electro.rename(columns={'DG1000420': 'LINESET', "2013-11-01 00:00": 'Timestamp', "37.439999" : 'Value Amp'})
#Riaggiungo il primo dato
electro=m_d.appforth(electro,['DG1000420','2013-11-01 00:00',37.439999])

temp = m_d.safe_import('SET-2')
temp=temp.rename(columns={'DG1000420': 'LINESET', "2013-12-01 00:00": 'Timestamp', "36.719997" : 'Value Amp'})
#Riaggiungo il primo dato
temp=m_d.appforth(temp,['DG1000420','2013-12-01 00:00',36.719997])

electro=electro.append(temp, ignore_index=True)
del temp  #Pesa tanto

electro

In [ ]:
#Unisco a dataframe che descrive posizioni sulla griglia
#Ne faccio un secondo che per alcune operazione è più comodo operare con l'altro
lines = m_d.safe_import('SET-lines')
electroLines = electro.merge(right=lines, how='outer')

#Ci sono incompatibilità tra i record salvati quindi li droppo cattivo
electroLines = electroLines.dropna()

electroLines

In [ ]:
#Serve svolgere il timestamp
temp=list(electro["Timestamp"])
electro["month"]=[int(st[5:7]) for st in temp]
electro["day"]=[int(st[8:10]) for st in temp]
#electro["hours"]=[int(st[11:13])+int(st[14:16])/60 for st in temp]
electro["hours"]=[np.around(int(st[11:13])+int(st[14:16])/60, 2) for st in temp]
    #Round per safer groupby, albeit slower (should not matter, just print uniques)
    #Utile che fa anche un file più leggero
#print(electro.hours.unique())
    
electro.drop(columns=["Timestamp"], inplace=True)
electro.to_csv(m_d.data_path_out / 'electro_final.csv',index=False)
    #Lo salvo che mi serve altrove, e questo è lenta da eseguire

electro

# CIRCOSCRIZIONI

In [ ]:
#Friendship ended with database del prof, database della provincia is now my best friend
circ=m_d.safe_import('circoscrizioni')

circ.plot("area")
circ=circ.sort_values("numero_cir").reset_index()
circ.drop(columns=["index", "numero_cir"], inplace=True)
circ

In [ ]:
#Sistemo per riallacciare
circ.drop(columns=["fumetto", "area", "perimetro"],inplace=True)
circ.rename(columns={"nome" : "circoscrizione"}, inplace=True)

# Unione di Databases
Passo 1: creare un database dei tweets; a ogni tweet associo tempo atmosferico e temperatura \

Il tempo viene binnato in 30 minuti (mediamente abbiamo sui 20 tweets all'ora), visto che i dati sono ottenuti ogni 10 o 15 perderei informazioni ma per i tweets uso diretto il valore dalla tabella)

In [ ]:
tweets=tweets.set_crs("EPSG:4326")
circ=circ.to_crs("EPSG:4326")
tweets = gpd.sjoin(tweets, grid, how="inner", op='intersects')
tweets.drop(columns="index_right", inplace=True)
tweets = gpd.sjoin(tweets, circ, how="left", op='intersects') 
    #Questi li attacco anche se potrei usare dirretto la grid, così è più comodo
tweets.drop(columns="index_right", inplace=True)
tweets


In [ ]:
#Visto che l'ora di tempo non è una quantità che correla a qualcosa di ciclico effettivamente misurabile*, ha poco senso
#separare ore e minuti ed quindi tanto vale trattarli insieme
#*Assumeremo eventi quali la campanella dei ragazzi delle superiori non influenzi in maniera importante i dati

###TEMPO DI ESECUZIONE QUALCHE MIN
from shapely.ops import nearest_points

Tw_final=pd.DataFrame()


N=len(tweets["created"])  #Numero tweets
temp=tweets["created"]
#TEMPO va smontato
Tw_final["month"]=[int(st[5:7]) for st in temp]
Tw_final["day"]=[int(st[8:10]) for st in temp]
Tw_final["hours"]=[int(st[11:13])+0.5*(int(st[14:16])>=30) for st in temp]

# TEMPERATURA E PRECIPITAZIONI
# per ricavare queste usiamo la stazione più vicina al tweet
# (sfortunamente il Trentino non ne ha tante, fortunatamente sono distribuite bene)
    
#Voglio farlo pythonico e rapido, quindi si userà nearest point
    #(ironicamente builtin sembra più lenta, si può ottimizzare ulteriormente)
#Vedi notebook note informative per una very very basic overview
nearest=[ stations[stations["geometry"]==nearest_points(gm,
            gpd.GeoSeries(stations["geometry"]).unary_union)[1]]["station"].values[0] for gm in tweets["geometry"]]
    #lista con nome della stazione più vicina

T=[]
R=[]
#Questo è ancora migliorabile come forma
for i in range(0,N):
    T.append(m_d.find_Weather(weather, Tw_final.loc[i]["month"],
                                Tw_final.loc[i]["day"], Tw_final.loc[i]["hours"], nearest[i], varType=0))
    R.append(m_d.find_Weather(weather, Tw_final.loc[i]["month"],
                                Tw_final.loc[i]["day"], Tw_final.loc[i]["hours"], nearest[i], varType=1))
Tw_final["temperature"]=T
Tw_final["rain"]=R


#Il resto lo importo direttamente
Tw_final["municipal"]=tweets["municipality.name"]
Tw_final["cellId"]=tweets["cellId"]
Tw_final["language"]=tweets["language"]
Tw_final["circoscrizione"]=tweets["circoscrizione"]

#Salviamo infine il database
Tw_final.to_csv(m_d.data_path_out / 'twitter_final.csv',index=False)

############MANCA ELETTRICITA' A QUESTO FILE (SE VOGLIO FARE EDA Ntw vs Elett)!!!!!!!!!!!

Tw_final

In [ ]:
#Roba che è meglio checckare
#Tw_final[Tw_final["temperature"].isnull()]  #Pochi tweets senza valore, poco male
Tw_final[Tw_final["circoscrizione"].isnull()]  #8000 tweets a trento, ok


In [ ]:
#Aggiungiamo anche il consumo elettrico associato al tweet
    #prob overkill per l'EDA ma dovrei saperlo fare in fretta ormai 
#Tw_final=pd.read_csv(m_d.data_path_out / "twitter_final.csv")

#ElectroLines.groupby("SQUAREID")


# Machine learning
Voglio creare un database su cui operare di machine learning \
Uso database salvati con appropriate assunzioni \
Lo faccio in funzione che è molto lunga e non ha c'è molto da presentare \
Vedi meglio in sezione ML \

In [ ]:
regressdB=m_d.df_reg()
regressdB.to_csv(m_d.data_path_out / 'MachineLearningDB.csv',index=False)

regressdB.head(5)